## FX Futures momentum strategy

## Pre-requisites

### 1. Open the example notebook in Colab
Our example notebooks assume you are using Google Colab. To open this notebook in Colab:
1. Sign-up or login to [Google Colab](https://colab.research.google.com/).
1. Select `File`.
1. Select `Open notebook`.
1. Select the `Github` tab.
1. Then:
    1. In `Enter a GitHub URL or search by organisation or user`, enter `SigTechnologies` and select the search option.
    1. In the `Repository` drop-down menu, select `SIGTechnologies/sigtech-python`.
    1. In the `branch` drop-down menu, select `master`.
1. Select the file you want to open.

### 2. Set up your Colab environment

In [ ]:
# Install our Python SDK
!pip install sigtech 

# Install ta as it is not included in Colab
!pip install ta

# Import os and our Python SDK
import os
import sigtech.api as sig

# Define your API key as a string. Remember to delete it before sharing your notebook with others. 
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import ta
import numpy as np

### 3. Start a session
After installing our python SDK, defining your API key, importing any Python libraries you require and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to a FX futures momentum strategy
An FX futures momentum strategy is a trading strategy that aims to capitalize on the persistence of trends in currency futures prices. The basic idea behind the strategy is to go long (buy) on currency futures that have exhibited strong positive returns over a certain period and go short (sell) on those with weak returns.

## Our strategy
- We will define a dictionary of FX assets and create a `RollingFutureStrategy` for each asset.
- For each contract's strategy we will calculate historical percentage changes (returns) over time and use these to generate signals for the strategy. After generating the signals, we will rank them in descending order for each contract using the rank method. The top 20% will be longed and the bottom 20% will be shorted.
- Finally, the ranked signals for each contract will be combined into a signal dataframe and input into the `SignalStrategy` class to create the trend following strategy.

## 1. Creating the assets

We define our currency futures assets in a dictionary mapping the contract codes to the contract sector for use in a function to generate multiple `RollingFutureStrategy`s.

In [ ]:
assets = {
    "AD": "CURNCY",
    "CD": "CURNCY",
    "SF": "CURNCY",
    "EC": "CURNCY",
    "BP": "CURNCY",
    "JY": "CURNCY",
    "NO": "CURNCY",
    "NV": "CURNCY",
    "SE": "CURNCY",
    "DX": "CURNCY"
}


In [ ]:
strategies = {}  

for contract_code, contract_sector in assets.items():
    strategy = sig.RollingFutureStrategy(
        contract_code=contract_code,
        contract_sector=contract_sector,
        currency='USD',
        start_date='2011-01-01',
        rolling_rule='front', # Rolls to the contract month with an expiration date closest to the current date.
        front_offset='-4:-1' # Specifies the number of business days before first delivery notice date (or expiry date for cash settled futures) to start and finish the roll.
    )
    strategies[contract_code] = strategy  # Store the generated strategies in a dictionary.



## 2. Generating signals
The function below is used to generate signals and rank them based on their performance. The top 20% will be assigned a value of 1 and signify when to take a long position, conversely the bottom 20% will be assigned a value of -1 and signify when to take a short position. The remaining signals will be given a value of 0 and signify when to be market-neutral.

In [ ]:
signals ={}

# For each strategy, calculate the percentage change in its historical performance over a 61-day period using the pct_change method applied to the strategy's historical data.
for asset, strategy in strategies.items():
    signals[strategy.name] = strategy.history().pct_change(61)

# Concatenate the signals from different strategies into a single DataFrame called signals_df.
signals_df = pd.concat(signals, join="inner", axis=1).dropna()

# Apply the rank method to signals_df. 
# axis=1 parameter means ranking is done horizontally (by date).
# method='max' means that if there are ties (signals with the same value), the highest rank will be assigned. 
# ascending=False parameter ensures that higher signal values receive higher ranks.
ranked_signals_df = signals_df.rank(axis=1, method='max', ascending = False)


final_rank_df = signals_df.copy()
pct = .20

top = round(pct*(len(ranked_signals_df.columns)))
bottom = (len(ranked_signals_df.columns) - top)

# For the top-ranked signals, set their values to 1 in final_rank_df, signalling a long position.
final_rank_df[ranked_signals_df <= top] = 1 

# For the bottom-ranked signals, set their values to -1 in final_rank_df, signalling a short position.
final_rank_df[ranked_signals_df > bottom] = -1

# For the signals falling between the top and bottom ranks, set their values to 0 in final_rank_df, signalling a neutral position.
final_rank_df[(ranked_signals_df >= top) & (ranked_signals_df <= bottom)] = 0

The following `.tail` codeblocks show how the original signal dataframe is transformed to a dataframe ranking the various signals and then into a dataframe which is compatible with `SignalStrategy`. 

> A `SignalStrategy` requires a signal dataframe as an input (this is a pandas dataframe where the column headers are the instrument names and the values are the signals for each of the instruments). These signals can be either a number of units *or* a weight.

In [ ]:
signals_df.tail(10)

In [ ]:
ranked_signals_df.tail(10)

In [ ]:
final_rank_df.tail(10)

## 3. Creating the FX futures momentum strategy

Our signal dataframe (`final_rank_df`) is used as the input for a `SignalStrategy` which rebalances at the end of each month.

In [ ]:
momentum_strategy = sig.SignalStrategy(
    signal_input=final_rank_df,
    currency = 'USD',
    start_date = final_rank_df.first_valid_index().date(),
    rebalance_frequency = 'EOM',
)

We can now get the historical performance of our strategy using `.history()` and plot these values using `.plot()`.

In [ ]:
momentum_strategy.history().plot(figsize=(20,10))